In [ ]:
# This notebook is to get the BIERs and PubMedBERT Dense (uniterpretable) numbers for Biosses 

# 1 Set BIER to use and load dependencies and data

In [ ]:
!pip install -q  transformers==2.5.1     #for BIER trained we need 2.5.1 transformers .. but for BertTokenClassifier use latest

     |████████████████████████████████| 499 kB 8.8 MB/s 
     |████████████████████████████████| 1.2 MB 50.6 MB/s 
     |████████████████████████████████| 132 kB 75.2 MB/s 
     |████████████████████████████████| 5.6 MB 21.5 MB/s 
     |████████████████████████████████| 880 kB 52.8 MB/s 
     |████████████████████████████████| 8.9 MB 57.1 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 138 kB 65.3 MB/s 
     |████████████████████████████████| 127 kB 57.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install -q faiss
!pip install -q faiss-gpu

     |████████████████████████████████| 4.7 MB 9.6 MB/s 
     |████████████████████████████████| 85.5 MB 1.4 MB/s 


In [ ]:
import copy
import csv
import datetime
import faiss
import gdown
import glob
import json
import numpy as np
import os
import pandas as pd
import random
import string
import time
import torch
import torch.nn as nn
from tqdm import tqdm
from random import shuffle

from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import get_linear_schedule_with_warmup
from transformers import AutoConfig
from transformers import BertModel, BertTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import AdamW

from sklearn import metrics, preprocessing

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## 1.1 Get BIOSSES data

In [ ]:
#from blurb/bluebert
def download_data(gid, out, quiet=True):
  gdown.download("https://drive.google.com/uc?id="+gid, out, quiet)

b1 = "biosses_all.tar"
b1id = "1DJ6r5cyKYItu9iqQMpsSLU1MRmsE63eE"
download_data(b1id, b1, quiet=True)

!tar xf biosses_all.tar
!ls BIOSSES/

dev.tsv  test.tsv  traindev.tsv  train.tsv


In [ ]:
!wc -l BIOSSES/train.tsv     #65
!wc -l BIOSSES/dev.tsv       #17     
!wc -l BIOSSES/traindev.tsv  #81
!wc -l BIOSSES/test.tsv      #21

65 BIOSSES/train.tsv
17 BIOSSES/dev.tsv
81 BIOSSES/traindev.tsv
21 BIOSSES/test.tsv


## 1.1.5 Look at Data

In [ ]:
import pandas as pd

train_df = pd.read_csv('BIOSSES/traindev.tsv', sep='\t')
test_df = pd.read_csv('BIOSSES/test.tsv', sep='\t')

print(train_df.shape, test_df.shape)

(80, 10) (20, 10)


## 1.2 Copy Model locally

In [1]:
# GET biomedical entity type system
!rm -rf ontology/
!mkdir ontology
!cd ontology && wget https://raw.githubusercontent.com/diegoolano/biomedical_interpretable_entity_representations/main/ier_model/ontology/medwiki_types_721_600k_full.txt
!ls ontology/

--2022-07-08 21:07:30--  https://raw.githubusercontent.com/diegoolano/biomedical_interpretable_entity_representations/main/ier_model/ontology/medwiki_types_721_600k_full.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1492327 (1.4M) [text/plain]
Saving to: ‘medwiki_types_721_600k_full.txt’

medwiki_types_721_6 100%[===================>]   1.42M  --.-KB/s    in 0.03s   

2022-07-08 21:07:31 (42.5 MB/s) - ‘medwiki_types_721_600k_full.txt’ saved [1492327/1492327]

medwiki_types_721_600k_full.txt


In [ ]:
bier_files = {
    "bier-blurb-pre":['blurb-pre/',
                      '11_01_e1_run_0720_600k_full_orig_microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_best.pt',
                        "1-3b2VRkGOkyoBGCm68_4d5m0aZGwQB0J"],
    }

def cp_model_from_drive(bfiles, mkdir=True):
  local_dir, save_as, gdrive_id = bfiles
  gdrive_file = "https://drive.google.com/uc?id=" + gdrive_id
  if mkdir:
    print("Making directory: ",local_dir)
    os.system("mkdir "+local_dir)
  print("Downloading File from Google Drive", gdrive_file, 
        "and saving to ", local_dir + save_as)
  gdown.download(gdrive_file, local_dir+save_as, quiet=False)

In [ ]:
!ls -laht blurb-pre/

def get_copy_of_orig_model():
  f = 'blurb-pre/11_01_e1_run_0720_600k_full_orig_microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_best.pt'
  os.system("cp "+f+" orig_bier_pubmedbert.pt")

def copy_over_orig_model():
  f = 'blurb-pre/11_01_e1_run_0720_600k_full_orig_microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_best.pt'
  os.system("cp orig_bier_pubmedbert.pt "+f)

get_copy_of_orig_model()
copy_over_orig_model()

!ls -laht blurb-pre/


ls: cannot access 'blurb-pre/': No such file or directory
ls: cannot access 'blurb-pre/': No such file or directory


#2 BIER

## 2.1 BIER Original model

In [ ]:
def get_transformer_model(model_type):
  #if model_type == 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext':
  return (AutoModel.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'), 
          AutoTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'))

class SimpleDecoder(nn.Module):
  def __init__(self, output_dim, answer_num):
    super(SimpleDecoder, self).__init__()
    self.answer_num = answer_num
    self.linear = nn.Linear(output_dim, answer_num, bias=False)

  def forward(self, inputs, output_type=None):
    output_embed = self.linear(inputs)
    return output_embed

class ModelBase(nn.Module):
  def __init__(self):
    super(ModelBase, self).__init__()
    self.loss_func = nn.BCEWithLogitsLoss()
    self.sigmoid_fn = nn.Sigmoid()

  def define_loss(self, logits, targets):
    loss = self.loss_func(logits, targets)
    return loss

  def forward(self, feed_dict):
    pass


#ORIGINAL
class TransformerModel(ModelBase):
  def __init__(self, args, answer_num):
    super(TransformerModel, self).__init__()
    print('In TransformerModel Initializing <{}> model...'.format(args.model_type))
    _model_class, _tokenizer_class = get_transformer_model(args.model_type)
    self.transformer_tokenizer = _tokenizer_class.from_pretrained(args.model_type)
    self.transformer_config = AutoConfig.from_pretrained(args.model_type)
    self.encoder = _model_class.from_pretrained(args.model_type)
    self.classifier = SimpleDecoder(self.transformer_config.hidden_size, answer_num)
    self.dropout = nn.Dropout(args.hidden_dropout_prob)
    self.avg_pooling = args.avg_pooling    

    self.env = args.env

  def forward(self, inputs, targets=None, labels=None):
    outputs = self.encoder(
      inputs["input_ids"],
      attention_mask=inputs["attention_mask"],
      token_type_ids=inputs["token_type_ids"] if "token_type_ids" in inputs else (None)
    )

    if self.avg_pooling:  # Averaging all hidden states
      outputs = (outputs[0] * inputs["attention_mask"].unsqueeze(-1)).sum(1)\
                / inputs["attention_mask"].sum(1).unsqueeze(-1)
    else:  # Use [CLS]
      outputs = outputs[0][:, 0, :]

    emb = outputs       
    logits = self.classifier(outputs)    #linear layer ( dot prods )
    sig_logits = self.sigmoid_fn(logits)

    if targets is not None:
      loss = self.define_loss(logits, targets)
    else:
      loss = None
    
    return loss, emb, logits, sig_logits


class TransformerModel_Classifier(ModelBase):
  # https://huggingface.co/transformers/_modules/transformers/models/bert/modeling_bert.html#BertForSequenceClassification
  def __init__(self, args, answer_num):
    super(TransformerModel_Classifier, self).__init__()
    print('In TransformerModel Classifier Initializing <{}> model...'.format(args.model_type))
    _model_class, _tokenizer_class = get_transformer_model(args.model_type)
    self.transformer_tokenizer = _tokenizer_class.from_pretrained(args.model_type)
    self.transformer_config = AutoConfig.from_pretrained(args.model_type)
    self.encoder = _model_class.from_pretrained(args.model_type)
    self.classifier = SimpleDecoder(self.transformer_config.hidden_size, answer_num)
    self.dropout = nn.Dropout(args.hidden_dropout_prob)
    self.avg_pooling = args.avg_pooling    

    self.env = args.env
    self.num_labels = args.num_labels
    self.freeze_bier = args.freeze_bier
    self.label_classifier = nn.Linear(answer_num, args.num_labels)

    # freeze encoder and type classifier layers ( comment out to not )
    if self.freeze_bier == True:
      for param in self.encoder.parameters():
        param.requires_grad = False
      for param in self.classifier.parameters():
        param.requires_grad = False
        
    elif self.freeze_bier == "except_bert":  #freeze all, except bert
      for param in self.encoder.parameters():
        param.requires_grad = False

    # allow label layer to learn new weights
    self.init_weights(self.label_classifier)

  def forward(self, inputs, targets=None, labels=None):
    outputs = self.encoder(
      inputs["input_ids"],
      attention_mask=inputs["attention_mask"],
      token_type_ids=inputs["token_type_ids"] if "token_type_ids" in inputs else (None)
    )

    if self.avg_pooling:  # Averaging all hidden states
      outputs = (outputs[0] * inputs["attention_mask"].unsqueeze(-1)).sum(1)\
                / inputs["attention_mask"].sum(1).unsqueeze(-1)
    else:  # Use [CLS]
      outputs = outputs[0][:, 0, :]

    emb = outputs
    outputs = self.dropout(outputs)         
    logits = self.classifier(outputs)    
    sig_logits = self.sigmoid_fn(logits)

    label_logits = self.label_classifier(sig_logits)
    loss = None # handle loss in training loop
    
    return loss, emb, label_logits, sig_logits

  def init_weights(self, module):
    """ Initialize the weights """
    print("In init weights: ", type(module))
    if isinstance(module, (nn.Linear, nn.Embedding)):
        # Slightly different from the TF version which uses truncated_normal for initialization
        # cf https://github.com/pytorch/pytorch/pull/5617
        module.weight.data.normal_(mean=0.0, std=0.02) 
    elif isinstance(module, BertLayerNorm):
        module.bias.data.zero_()
        module.weight.data.fill_(1.0)
    if isinstance(module, nn.Linear) and module.bias is not None:
        module.bias.data.zero_()

## 2.2 Helper BIER functions

In [ ]:
def load_model(reload_model_name, save_dir, model, optimizer_enc=None, optimizer_cls=None):
  model_file_name = '{0:s}/{1:s}.pt'.format(save_dir, reload_model_name)  
  checkpoint = torch.load(model_file_name)
  print("Loading ", model_file_name)
  
  if len(model.state_dict()) != len(checkpoint['state_dict']):
    print("Size expected vs state_dict size passed in ", len(model.state_dict()), len(checkpoint['state_dict']))
    
  model.load_state_dict(checkpoint['state_dict'], strict=False)  #false to allow for new classifier layer
  
  if optimizer_enc and optimizer_cls:  # Continue training
    print("CHECKPOINT Optim Enc", checkpoint['optimizer_enc'].keys())
    optimizer_enc.load_state_dict(checkpoint['optimizer_enc'])  
    optimizer_cls.load_state_dict(checkpoint['optimizer_cls'])
  else:  # Test
    total_params = 0    
    for k in checkpoint['state_dict']:
      elem = checkpoint['state_dict'][k]
      param_s = 1
      for size_dim in elem.size():
        param_s = size_dim * param_s    
      total_params += param_s

    param_str = ('Number of total parameters..{0:d}'.format(total_params))
    print(param_str)

  print('Loading model from ... {0:s}'.format(model_file_name))


def load_vocab_dict(vocab_file_name, vocab_max_size=None, start_vocab_count=None, common_vocab_file_name=None):
  with open(vocab_file_name) as f:
    text = [x.strip() for x in f.readlines()]
    if vocab_max_size:
      text = text[:vocab_max_size]
    if common_vocab_file_name:
        print('==> adding common training set types')
        print('==> before:', len(text))
        with open(common_vocab_file_name, 'r') as fc:
            common = [x.strip() for x in fc.readlines()]
        print('==> common:', len(common))
        text = list(set(text + common))
        print('==> after:', len(text))
    if start_vocab_count:
      file_content = dict(zip(text, range(0 + start_vocab_count, len(text) + start_vocab_count)))
    else:
      file_content = dict(zip(text, range(0, len(text))))
  return file_content


def get_which_bert_and_ontology_type(env):
  which_bert = "bert"
  if "biobert" in env:
    which_bert = "biobert"
  elif "scibert" in env:
    which_bert = "scibert"
  elif "blurb" in env:
    which_bert = "blurb"

  if env == "yasu":
    ontology_type = "60k"
  elif "-3k" in env:
    ontology_type = "bier-3k-pre"
    if "-expand" in env:
      ontology_type = "bier-3k-expand"
  else:
    ontology_type = "bier-600k-pre"
    if "-expand" in env:
      ontology_type = "bier-600k-expand"

  return which_bert, ontology_type

### CONSTANTS 
BASE_PATH = "."
ONTOLOGY_DICT = {
	'bier-600k-pre': BASE_PATH +"/ontology/medwiki_types_721_600k_full.txt",	  
}     
ANS2ID_DICT_MEDWIKI_720_600ko = load_vocab_dict(ONTOLOGY_DICT["bier-600k-pre"]) 
ANSWER_NUM_DICT = {'bier-600k-pre':len(ANS2ID_DICT_MEDWIKI_720_600ko)}      
ANS2ID_DICT = {'bier-600k-pre': ANS2ID_DICT_MEDWIKI_720_600ko}    
ID2ANS_DICT_MEDWIKI_720_600ko = {v: k for k, v in ANS2ID_DICT_MEDWIKI_720_600ko.items()}
ID2ANS_DICT = {'bier-600k-pre': ID2ANS_DICT_MEDWIKI_720_600ko}

def get_topk_types(prob, BERT_TO_USE, k=100):
  which_bert, ontology_type = get_which_bert_and_ontology_type(env=BERT_TO_USE)
  id2ans = ID2ANS_DICT[ontology_type]
  #print("Getting topk with ", ontology_type)
  #print("Sample Probs: ",[(p, id2ans[i]) for i, p in enumerate(prob) if i < 10 and i > 0])

  if "600k" in ontology_type: #env == "0720_600k_full":
    sorted_prob = sorted( [(p, id2ans[i]) for i, p in enumerate(prob) if i != 0], key=lambda x: x[0], reverse=True)  #instead of id2ans[i+1]
  else:
    sorted_prob = sorted( [(p, id2ans[i]) for i, p in enumerate(prob)], key=lambda x: x[0], reverse=True)

  return sorted_prob[:k] + [(k, v) for k, v in sorted_prob if k <= 0.]

##2.3 Load saved model into BIER and set BERT type

In [ ]:
#del model
torch.cuda.empty_cache()

In [ ]:
def load_bier_model(BERT_TO_USE, is_wiki_desc=False, num_labels=False, freeze_bier=False):

  env = BERT_TO_USE
  model_path = bier_files[BERT_TO_USE][0]
  goal = "medwiki" if env != "yasu" else "60k"

  if is_wiki_desc:
    reload_model_name = wiki_desc_files[BERT_TO_USE][1].replace(".pt","")
  else:
    reload_model_name = bier_files[BERT_TO_USE][1].replace(".pt","")

  which_bert, ontology_type = get_which_bert_and_ontology_type(env)

  btype = "WikiDesc" if is_wiki_desc else "BierEntityType"
  print("LOADING:", btype, env, model_path, reload_model_name, goal, which_bert, ontology_type)
  
  modeltype = {'blurb': 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'}

  from types import SimpleNamespace
  args_dict = {}
  args_dict['device'] = device
  args_dict['task'] = 'ehr'
  args_dict['env'] = env 
  args_dict['reload_model_name'] = reload_model_name
  args_dict['which_bert'] = which_bert
  args_dict['model_type'] = modeltype[which_bert]
  args_dict['goal'] = goal
  args_dict['max_position_embeddings'] = 512
  args_dict['do_lower'] = True
  args_dict["hidden_dropout_prob"] = .1 
  args_dict["avg_pooling"] = False

  if num_labels != False:
    args_dict['num_labels'] = num_labels
    args_dict['freeze_bier'] = freeze_bier
    
  args_dict['model_path'] = model_path
  args_dict['mode'] =  "test"  
  args_dict["local_rank"] = -1
  args_dict["context_window_size"] = 100
  args_dict["per_gpu_train_batch_size"] = 8
  args_dict["per_gpu_eval_batch_size"] = 8,
  args_dict["warmup_steps"] =0
  args_dict["threshold"] =0.5
  args_dict["learning_rate_cls"] = 1e-3
  args_dict["learning_rate_enc"] = 2e-5
  args_dict["adam_epsilon_enc"] = 1e-8
  args_dict["adam_epsilon_cls"] = 1e-8
  args_dict["gradient_accumulation_steps"] = 4
  args_dict["seed"] = 42
  args_dict["n_gpu"] = 1  
  args_dict["load"] = True
  args_dict["eval_batch_size"] = 1 

  args = SimpleNamespace(**args_dict)

  if num_labels == False:
    print("Load BIER Transformer Model", args.env, args.goal, ontology_type)
    model = TransformerModel(args, ANSWER_NUM_DICT[ontology_type])
  else:
    print("Load BIER Transformer Classifier Model", args.env, args.goal, ontology_type)
    model = TransformerModel_Classifier(args, ANSWER_NUM_DICT[ontology_type])

  print("DONE LOADING framework", type(model), "with size", len(model.state_dict()) ) 
  

  # load encoder/classifier params
  if num_labels == False:
    no_decay = ["bias", "LayerNorm.weight"]
    classifier_param_name = ["classifier.linear.weight"]
    encoder_parameters = [
      {"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay) and n not in classifier_param_name],
        "weight_decay": 0.0  },
      {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay) and n not in classifier_param_name],
        "weight_decay": 0.0},
    ]
    classifier_parameters = [
      { "params": [p for n, p in model.named_parameters() if n in classifier_param_name],
        "weight_decay": 0.0},
    ]
    print(
      'Encoder {}, Classifier {}'.format(
        sum([len(p['params']) for p in encoder_parameters]),
        sum([len(p['params']) for p in classifier_parameters])
      )
    )

    print("Encoder Params", len(encoder_parameters[0]["params"]), len(encoder_parameters[1]["params"]))
    optimizer_enc = AdamW(encoder_parameters, lr=args.learning_rate_enc, eps=args.adam_epsilon_enc)
    optimizer_cls = AdamW(classifier_parameters, lr=args.learning_rate_cls, eps=args.adam_epsilon_cls)

    load_model(args.reload_model_name, args.model_path, model, optimizer_enc, optimizer_cls)  #<-- do i need to do this?
    optimizer_enc.zero_grad()
    optimizer_cls.zero_grad()
  else:
    load_model(args.reload_model_name, args.model_path, model) 

  return model, args

## 2.4 Auxiliary Bier Task functions

In [ ]:
def input_to_torch(batch, device):
  inputs_to_model = {k: v.to(device) for k, v in batch['inputs'].items()}
  return inputs_to_model

def get_embedding_and_types(ex, model, tokenizer = None, max_len=512):  
  if tokenizer == None:
    context_tokens = model.transformer_tokenizer.encode_plus(ex[0],ex[1])
    cur_len = len(context_tokens['input_ids'])
    inputs = model.transformer_tokenizer.encode_plus(    
          ex[0],ex[1],
          add_special_tokens=True,
          max_length=min(max_len, cur_len),
          truncation_strategy='only_second',
          pad_to_max_length=True,
          return_tensors='pt'
        )        
    batch = {'inputs':inputs}
    inputs_to_model = input_to_torch(batch, device)  
    _, emb, logits, sig_logits = model.forward(inputs_to_model)   #CLS embedding, TASK logits,  IER sparse logits
    return emb, logits, sig_logits, 
  else:
    # for non-bier load
    context_tokens = tokenizer.encode_plus(mention,abstract_txt)
    cur_len = len(context_tokens['input_ids'])
    inputs = tokenizer.encode_plus(    
          mention,abstract_txt,
          add_special_tokens=True,
          max_length=min(max_len, cur_len),
          truncation_strategy='only_second',
          pad_to_max_length=True,
          return_tensors='pt'
        )
    batch = {'inputs':inputs}
    inputs_to_model = input_to_torch(batch, device)  
    encoded_layers, logits = model(
                            input_ids = inputs_to_model['input_ids'], 
                            token_type_ids = inputs_to_model['token_type_ids'], 
                            attention_mask = inputs_to_model['attention_mask'])

    vec = encoded_layers[0][0][:]   
    vec = vec.detach().cpu().numpy()
    return vec, logits
  

"""
pmid = list(train_data.keys())[0]
exs = train_data[pmid] # title, abstract, mentions

# pmid:  PMID-19406993
# ex[0]:  Erbb2 suppresses DNA damage-induced checkpoint activation and UV-induced mouse skin tumorigenesis. 
# ex[1]:  The Erbb2 receptor is activated by UV irradiation, the primary cause of non-melanoma skin cancer. We hypothesized that Erbb2 activation contributes to UV-induced skin tumorigenesis by suppressing cell cycle arrest. Consistent with this hypothesis, inhibition of Erbb2 in v-ras(Ha) transgenic mice before UV exposure resulted in both 56% fewer skin tumors and tumors that were 70% smaller. Inhibition of the UV-induced activation of Erbb2 also resulted in milder epidermal hyperplasia, S-phase accumulation, and decreased levels of the cell cycle regulator Cdc25a, suggesting altered cell cycle regulation on inhibition of Erbb2. Further investigation using inhibition or genetic deletion of Erbb2 in vitro revealed reduced Cdc25a levels and increased S-phase arrest in UV-irradiated cells lacking Erbb2 activity. Ectopic expression of Cdc25a prevented UV-induced S-phase arrest in keratinocytes lacking Erbb2 activity, demonstrating that maintenance of Cdc25a by Erbb2 suppresses cell cycle arrest. Examination of checkpoint pathway activation upstream of Cdc25a revealed Erbb2 activation did not alter Ataxia Telangiectasia and Rad3-related/Ataxia Telangiectasia Mutated activity but increased inhibitory phosphorylation of Chk1-Ser(280). Since Akt phosphorylates Chk1-Ser(280), the effect of Erbb2 on phosphatidyl inositol-3-kinase (PI3K)/Akt signaling during UV-induced cell cycle arrest was determined. Erbb2 ablation reduced the UV-induced activation of PI3K while inhibition of PI3K/Akt increased UV-induced S-phase arrest. Thus, UV-induced Erbb2 activation increases skin tumorigenesis through inhibitory phosphorylation of Chk1, Cdc25a maintenance, and suppression of S-phase arrest via a PI3K/Akt-dependent mechanism.
# ex[2]:  ['T1', 'Gene_or_gene_product 0 5', 'Erbb2']   |  id, label, mention

embedding, logits, sig_logits = get_embedding_and_types(ex, model)
"""

#then use BIER embeddings and FAISS for train/dev/test sets to get embeddings and store pair wises distances
print()

In [ ]:
def show_types_for(logits, id2ans, k=100):  
  ## see id_ work on micro for faster way to to this and topk
  prob = logits.detach().cpu().numpy().flatten()
  if 0 not in list(id2ans.keys()):
    prob_word_pairs = [(p, id2ans[i]) for i, p in enumerate(prob) if i != 0]    
  else:
    prob_word_pairs = [(p, id2ans[i]) for i, p in enumerate(prob)]

  sorted_prob = sorted( prob_word_pairs, key=lambda x: x[0], reverse=True)
  return sorted_prob[:k] + [(k, v) for k, v in sorted_prob if k <= 0.]

print()

## 2.5 Do BIER runs

In [ ]:
torch.cuda.empty_cache()
!nvidia-smi

Tue Jun 28 14:38:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
print("Select MODEL to use from the following: ", list(bier_files.keys()))
BERT_TO_USE = 'bier-blurb-pre' 

Select MODEL to use from the following:  ['bier-blurb-pre']


In [ ]:
which_bert, ontology_type = get_which_bert_and_ontology_type(env=BERT_TO_USE)
bier_id2ans = ID2ANS_DICT[ontology_type]       #for top types
bier_word2id = ANS2ID_DICT[ontology_type]       #for type to id
bier_ansnum = ANSWER_NUM_DICT[ontology_type]

In [ ]:
train_df.head(2)

,index,genre,filename,year,old_index,source1,source2,sentence1,sentence2,score
0,0,GENRE,filename,1997,1,BIOSSES,BIOSSES,"Here, looking for agents that could specifical...","Not surprisingly, GATA2 knockdown in KRAS muta...",2.2
1,1,GENRE,filename,1997,1,BIOSSES,BIOSSES,MLL-FKBP and MLL-AF9 transformed cells showed ...,Regardless of the mechanism for transcriptiona...,3.2


## 2.6 Fine Tune BIER Sparse for Regression


In [ ]:
# Set Batch Size and Epochs to run


def get_data_loader(df, model, batch_size=8):
  #df can be train_df or test_df
  st = time.time()
  max_len = 512
  train_input_ids, train_attention_masks = [], []
  train_labels = []
  train_info = []

  for id in list(df["index"].values):
    row = df[df["index"] == id]
    ex = [row.sentence1.values[0], row.sentence2.values[0], row.score.values[0]]
    torch.cuda.empty_cache()
    context_tokens = model.transformer_tokenizer.encode_plus(ex[0],ex[1])
    cur_len = len(context_tokens['input_ids'])
    encoded_dict = model.transformer_tokenizer.encode_plus(    
          ex[0],ex[1],
          add_special_tokens=True,        
          max_length=max_len,
          truncation_strategy='only_second',
          pad_to_max_length=True,
          return_tensors='pt'
        )
    train_input_ids.append(encoded_dict['input_ids'][0])
    train_attention_masks.append(encoded_dict['attention_mask'][0]) 
    train_labels.append(ex[2])
    train_info.append([id,ex[0]])
  print("Elapsed.", time.time() - st)     #119

  # Convert the lists into PyTorch tensors.
  train_pt_input_ids = torch.stack(train_input_ids, dim=0)
  train_pt_attention_masks = torch.stack(train_attention_masks, dim=0)
  train_pt_labels = torch.tensor(train_labels)  

  # Create the DataLoader for our training set.
  train_data_set = TensorDataset(train_pt_input_ids, train_pt_attention_masks, train_pt_labels)
  train_sampler = RandomSampler(train_data_set)
  train_dataloader = DataLoader(train_data_set, sampler=train_sampler, batch_size=batch_size)
  return train_dataloader

In [ ]:
import datetime
import random
import time
from transformers import get_linear_schedule_with_warmup


def format_time(elapsed):
    # Takes a time in seconds and returns a string hh:mm:ss
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def train_model( model, train_dataloader, test_prediction_dataloader, device, learning_rate=5e-5, epochs=30):

  optimizer = AdamW(model.parameters(),
                    lr = learning_rate, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                    eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )

  # Total number of training steps is number of batches * number of epochs.
  total_steps = len(train_dataloader) * epochs

  # Create the learning rate scheduler.
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps)
  seed_val = 42
  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)
  loss_values = []
  tot_time = 0

  for epoch_i in range(0, epochs):
      t0 = time.time()
      total_loss = 0
      model.train()

      for step, batch in enumerate(train_dataloader):
          if step % 100 == 0 and not step == 0:
              elapsed = format_time(time.time() - t0)
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)
          b_labels = b_labels.to(torch.float32) 

          torch.cuda.empty_cache()
          model.zero_grad()        

          inputs = {'input_ids': b_input_ids, 'token_type_ids': None, 'attention_mask': b_input_mask}
          outputs = model(inputs, labels=b_labels)  #loss, emb, label_logits, sig_logits ( using BIER )
          
          loss_fct = nn.MSELoss()   #  incorrect for regression 
          _, cls_emb, label_logits, ier_logits = outputs
          loss = loss_fct(label_logits.squeeze(), b_labels.squeeze())

          total_loss += loss.item()
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          optimizer.step()
          scheduler.step()

      avg_train_loss = total_loss / len(train_dataloader)            
      loss_values.append(avg_train_loss)
      
      print('Epoch ',epoch_i + 1,' / ',epochs,' elapsed:', format_time(time.time() - t0), ', loss: ', avg_train_loss)       
      mse, corr, t_all_predictions = do_eval(model, test_prediction_dataloader, device, print_out=False)
      print("---- eval mse:",mse, "and pearsons corr:", corr)
      tot_time += time.time() - t0
      

  print("")
  print("Training complete! Total elapsed:", format_time(tot_time)   )

In [ ]:
#Eval test 
from scipy.stats import pearsonr

def do_eval(model, test_prediction_dataloader, device, print_out=True):
  if print_out:
    print("Evaling ", BERT_TO_USE)
  test_predictions , test_true_labels = [], []

  model.eval()
  for tbatch in test_prediction_dataloader:
    tbatch = tuple(t.to(device) for t in tbatch)
    test_b_input_ids, test_b_input_mask, test_b_labels = tbatch
    test_b_labels = test_b_labels.to(torch.float32)

    with torch.no_grad():
        torch.cuda.empty_cache()

        test_inputs = {'input_ids': test_b_input_ids, 'token_type_ids': None, 'attention_mask': test_b_input_mask}
        test_outputs = model(test_inputs)      #loss, emb, label_logits, sig_logits ( using BIER ) 

        test_logits = test_outputs[2]   #this may need to be test_outputs[2] and not 0?
        test_logits = test_logits.detach().cpu().numpy()
    
        test_label_ids = test_b_labels.to('cpu').numpy()
        test_predictions.append(test_logits)
        test_true_labels.append(test_label_ids)
    
  # call eval
  t_all_predictions = np.concatenate(test_predictions, axis=0)
  t_all_true_labels = np.concatenate(test_true_labels, axis=0)

  errs_sqd = []
  for i in range(t_all_predictions.shape[0]):
    v = t_all_predictions[i][0] - t_all_true_labels[i]
    if print_out:
      print(i,t_all_predictions[i][0], t_all_true_labels[i], v*v)
    errs_sqd.append(v*v)

  mse = sum(errs_sqd)/len(errs_sqd)
  corr, _ = pearsonr(t_all_predictions.squeeze(), t_all_true_labels)
  if print_out:
    print("MSE: ", mse )
    print('Pearsons correlation: %.3f' % corr)

  return mse, corr, t_all_predictions

In [ ]:
#Below Set the hyper params you want to run over ( learning_rate, type of freezing, epochs, etc)

In [ ]:
def do_entire_run(device, cp_files_over=False, freeze_bier = True, learning_rate=5e-5, epochs=30, batch_size=8):
  t1 = time.time()
  BERT_TO_USE =  'bier-blurb-pre'
  if cp_files_over:
    print("--Copying over BIER Model")
    cp_model_from_drive(bier_files[BERT_TO_USE])
    get_copy_of_orig_model()   #cp orig_model (longname) to bier_pubmedbert.pt
  else:
    copy_over_orig_model()  ##cp bier_pubmedbert.pt to orig_model (longname)

  # Load BIER Model ( ie, left hand mention/context entity encoder)

  torch.cuda.empty_cache()
  is_wiki_desc=False
  num_labels=1
  model, args = load_bier_model(BERT_TO_USE, is_wiki_desc, num_labels, freeze_bier)
  model.to(device)
  model.cuda()


  train_dataloader = get_data_loader(train_df, model, batch_size)
  test_prediction_dataloader = get_data_loader(test_df, model, batch_size)

  train_model( model, train_dataloader, test_prediction_dataloader, device, learning_rate, epochs)
  mse, corr, t_all_predictions = do_eval(model, test_prediction_dataloader, device)

  print("Total elapsed:", format_time(time.time() - t1))  #0:01:23 for 30 epochs frozen
  return mse, corr, t_all_predictions


# For all datasets, we use the development set for tuning the hyperparameters with the same range: 
# learning rate (1e-5, 3e-5, 5e-5), batch size (16, 32) and epoch number (2–60)
lrs = [7e-6, 9e-6, 1e-5, 2e-5, 3e-5, 4e-5, 5e-5, 7e-5]

freeze_types = [True, False,"except_bert"]
cp_files_over = True
epochs = 100
batch_size = 8
results = []
t0 = time.time()
for learning_rate in lrs:
  for freeze_type in freeze_types:
    ti = time.time()
    print("!!!!! DOING RUN FOR ", "LR:",learning_rate, "Freeze:",freeze_type, "Eps:", epochs)
    if len(results) > 0:
      cp_files_over = False
    mse, corr, t_all_predictions = do_entire_run(device, cp_files_over, freeze_type, learning_rate, epochs, batch_size)
    results.append([learning_rate,freeze_type,epochs,batch_size,mse,corr,format_time(time.time() - ti), t_all_predictions])
print("Done. Overall runs elapsed:", format_time(time.time() - t0))

for r in results:
  print(r[0:4],round(r[4],4), round(r[5],4),r[6:8])

"""
# LR   Freeze, EP, Bt, MSE , 
[7e-06, True, 30,  8] 5.252  
[9e-06, True, 100, 8] 5.2568 
[1e-05, True, 30,  8] 5.2779 
[2e-05, True, 100, 8] 5.1822 
[3e-05, True, 30,  8] 5.2037 
[4e-05, True, 100, 8] 5.0495   <-- best MSE RESULTS
[5e-05, True, 30,  8] 5.162  
[7e-05, True, 30,  8] 5.1207 
"""
print()

!!!!! DOING RUN FOR  LR: 5e-05 Freeze: True Eps: 100
--Copying over BIER Model
Making directory:  blurb-pre/


Downloading...
From: https://drive.google.com/uc?id=1-3b2VRkGOkyoBGCm68_4d5m0aZGwQB0J
To: /content/blurb-pre/11_01_e1_run_0720_600k_full_orig_microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_best.pt
100%|██████████| 1.94G/1.94G [00:22<00:00, 86.4MB/s]


LOADING: BierEntityType bier-blurb-pre blurb-pre/ 11_01_e1_run_0720_600k_full_orig_microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_best medwiki blurb bier-600k-pre
Load BIER Transformer Classifier Model bier-blurb-pre medwiki bier-600k-pre
In TransformerModel Classifier Initializing <microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext> model...


Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In init weights:  <class 'torch.nn.modules.linear.Linear'>
DONE LOADING framework <class '__main__.TransformerModel_Classifier'> with size 202
Loading  blurb-pre//11_01_e1_run_0720_600k_full_orig_microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_best.pt
Size expected vs state_dict size passed in  202 200
Number of total parameters..161939712
Loading model from ... blurb-pre//11_01_e1_run_0720_600k_full_orig_microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_best.pt
Elapsed. 0.2693920135498047
Elapsed. 0.058173418045043945


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


Epoch  1  /  100  elapsed: 0:00:03 , loss:  6.501072931289673
---- eval mse: 5.314809561825177 and pearsons corr: 0.29623389782076726
Epoch  2  /  100  elapsed: 0:00:03 , loss:  6.486633682250977
---- eval mse: 5.307742991671148 and pearsons corr: 0.29073514257835326
Epoch  3  /  100  elapsed: 0:00:03 , loss:  6.482024621963501
---- eval mse: 5.300301053970907 and pearsons corr: 0.28710584923338406
Epoch  4  /  100  elapsed: 0:00:03 , loss:  6.467746257781982
---- eval mse: 5.293465793953146 and pearsons corr: 0.28171955971573015
Epoch  5  /  100  elapsed: 0:00:03 , loss:  6.466936635971069
---- eval mse: 5.28670408400576 and pearsons corr: 0.2765952222665917
Epoch  6  /  100  elapsed: 0:00:03 , loss:  6.463567209243775
---- eval mse: 5.280219173031947 and pearsons corr: 0.2728510842666778
Epoch  7  /  100  elapsed: 0:00:03 , loss:  6.443944263458252
---- eval mse: 5.273632629669737 and pearsons corr: 0.26751688561351517
Epoch  8  /  100  elapsed: 0:00:03 , loss:  6.437437391281128
---

In [ ]:
for r in results:
  print(r[0:4],round(r[4],4), round(r[5],4), r[6])

## 2.4  Fine Tune PubMedBert baseline

In [ ]:
# https://medium.com/@anthony.galtier/fine-tuning-bert-for-a-regression-task-is-a-description-enough-to-predict-a-propertys-list-price-cf97cd7cb98a

model_type = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'  
model_class = AutoModel.from_pretrained(model_type)
tokenizer_class = AutoTokenizer.from_pretrained(model_type)
torch.cuda.empty_cache()
transformer_tokenizer = tokenizer_class.from_pretrained(model_type)
pmbert_encoder = model_class.from_pretrained(model_type)
pmbert_encoder.to(device)
pmbert_encoder.train()
print()

Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
""" maybe if performance is baaad
from sklearn.preprocessing import StandardScaler
price_scaler = StandardScaler()
price_scaler.fit(train_labels.reshape(-1, 1))
train_labels = price_scaler.transform(train_labels.reshape(-1, 1))
test_labels = price_scaler.transform(test_labels.reshape(-1, 1))
"""

In [ ]:
#extend model to do Regression 
class PubMedBertRegressor(nn.Module):

    def __init__(self, model, drop_rate=0.2, freeze_bert=False):
        super(PubMedBertRegressor, self).__init__()
        D_in, D_out = 768, 1      
        self.pubmedbert = model
        self.regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(D_in, D_out))
        
    def forward(self, input_ids, attention_masks):
        
        outputs = self.pubmedbert(input_ids, attention_masks)
        class_label_output = outputs[1]
        outputs = self.regressor(class_label_output)
        return outputs

regressor_model = PubMedBertRegressor(pmbert_encoder, drop_rate=0.2)
regressor_model.to(device)
print()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import datetime
import random
import time
from transformers import get_linear_schedule_with_warmup


# Set Batch Size, Num Epochs and LR to use for run
batch_size = 8
epochs = 60   
learning_rate = 2e-5

#load training + dev data
st = time.time()
max_len = 512
train_input_ids, train_attention_masks = [], []
train_labels = []
train_info = []

for id in list(train_df["index"].values):
  row = train_df[train_df["index"] == id]
  ex = [row.sentence1.values[0], row.sentence2.values[0], row.score.values[0]]
  torch.cuda.empty_cache()
  context_tokens = transformer_tokenizer.encode_plus(ex[0],ex[1])
  cur_len = len(context_tokens['input_ids'])
  encoded_dict = transformer_tokenizer.encode_plus(    
        ex[0],ex[1],
        add_special_tokens=True,        
        max_length=max_len,
        truncation_strategy='only_second',
        pad_to_max_length=True,
        return_tensors='pt'
      )
  train_input_ids.append(encoded_dict['input_ids'][0])
  train_attention_masks.append(encoded_dict['attention_mask'][0]) 
  train_labels.append(ex[2])
  train_info.append([id,ex[0]])
print("Elapsed.", time.time() - st)     #119

# Convert the lists into PyTorch tensors.
train_pt_input_ids = torch.stack(train_input_ids, dim=0)
train_pt_attention_masks = torch.stack(train_attention_masks, dim=0)
#train_pt_labels = torch.tensor(train_labels, dtype=torch.long)  
train_pt_labels = torch.tensor(train_labels)  

# Create the DataLoader for our training set.
train_data_set = TensorDataset(train_pt_input_ids, train_pt_attention_masks, train_pt_labels)
train_sampler = RandomSampler(train_data_set)
train_dataloader = DataLoader(train_data_set, sampler=train_sampler, batch_size=batch_size)

torch.cuda.empty_cache()

optimizer = AdamW(regressor_model.parameters(),
                  lr = learning_rate, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

def format_time(elapsed):
    # Takes a time in seconds and returns a string hh:mm:ss
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
loss_values = []

loss_function = nn.MSELoss()

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0
    regressor_model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_labels = b_labels.to(torch.float32) 

        torch.cuda.empty_cache()
        regressor_model.zero_grad()        

        #inputs = {'input_ids': b_input_ids, 'token_type_ids': None, 'attention_masks': b_input_mask}
        outputs = regressor_model(b_input_ids, b_input_mask)  
        
        if step == 0 and epoch == 0:
          print(type(b_input_ids), b_input_ids.shape)
          print(type(b_input_mask), b_input_mask.shape)
          print(type(outputs), outputs.shape)
          print(type(b_labels), b_labels.shape)

        loss = loss_function(outputs.squeeze(), b_labels.squeeze())
              
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(regressor_model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)            
    loss_values.append(avg_train_loss)

    print("\n  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))        
    t0 = time.time()

print("")
print("Training complete!")  

Elapsed. 0.2657437324523926

======== Epoch 1 / 60 ========
Training...
<class 'torch.Tensor'> torch.Size([8, 512])
<class 'torch.Tensor'> torch.Size([8, 512])
<class 'torch.Tensor'> torch.Size([8, 1])
<class 'torch.Tensor'> torch.Size([8])


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)



  Average training loss: 3.26
  Training epcoh took: 0:00:06

======== Epoch 2 / 60 ========
Training...
<class 'torch.Tensor'> torch.Size([8, 512])
<class 'torch.Tensor'> torch.Size([8, 512])
<class 'torch.Tensor'> torch.Size([8, 1])
<class 'torch.Tensor'> torch.Size([8])

  Average training loss: 1.33
  Training epcoh took: 0:00:06

======== Epoch 3 / 60 ========
Training...
<class 'torch.Tensor'> torch.Size([8, 512])
<class 'torch.Tensor'> torch.Size([8, 512])
<class 'torch.Tensor'> torch.Size([8, 1])
<class 'torch.Tensor'> torch.Size([8])

  Average training loss: 1.15
  Training epcoh took: 0:00:06

======== Epoch 4 / 60 ========
Training...
<class 'torch.Tensor'> torch.Size([8, 512])
<class 'torch.Tensor'> torch.Size([8, 512])
<class 'torch.Tensor'> torch.Size([8, 1])
<class 'torch.Tensor'> torch.Size([8])

  Average training loss: 0.78
  Training epcoh took: 0:00:06

======== Epoch 5 / 60 ========
Training...
<class 'torch.Tensor'> torch.Size([8, 512])
<class 'torch.Tensor'> to

In [ ]:
st = time.time()
regressor_model.eval()
test_input_ids, test_attention_masks = [], []
test_labels = []
test_info = []

#for i, pmid in enumerate(test_pmids):
for id in list(test_df["index"].values):
  row = test_df[test_df["index"] == id]
  ex = [row.sentence1.values[0], row.sentence2.values[0], row.score.values[0]]
  torch.cuda.empty_cache()
  context_tokens = transformer_tokenizer.encode_plus(ex[0],ex[1])
  cur_len = len(context_tokens['input_ids'])
  encoded_dict = transformer_tokenizer.encode_plus(    
        ex[0],ex[1],
        add_special_tokens=True,        
        max_length=max_len,
        truncation_strategy='only_second',
        pad_to_max_length=True,
        return_tensors='pt'
      )
  test_input_ids.append(encoded_dict['input_ids'][0])
  test_attention_masks.append(encoded_dict['attention_mask'][0]) 
  test_labels.append(ex[2])
  test_info.append([id,ex[0]])


# Convert the lists into PyTorch tensors.
test_pt_input_ids = torch.stack(test_input_ids, dim=0)
test_pt_attention_masks = torch.stack(test_attention_masks, dim=0)
#test_pt_labels = torch.tensor(test_labels, dtype=torch.long)  
test_pt_labels = torch.tensor(test_labels)  

print(test_labels[0:5])
print(test_pt_labels[0:5])

# Set the batch size and # Create the DataLoader for our testing set.
test_batch_size = 1
test_prediction_data = TensorDataset(test_pt_input_ids, test_pt_attention_masks, test_pt_labels)
test_prediction_sampler = SequentialSampler(test_prediction_data)
test_prediction_dataloader = DataLoader(test_prediction_data, sampler=test_prediction_sampler, batch_size=test_batch_size)
print("Elapsed.", time.time() - st)   #50 seconds


#Eval test 
print("Evaling ")
test_predictions , test_true_labels, test_loss = [], [], []

for tbatch in test_prediction_dataloader:
  tbatch = tuple(t.to(device) for t in tbatch)
  test_b_input_ids, test_b_input_mask, test_b_labels = tbatch
  test_b_labels = test_b_labels.to(torch.float32)
  with torch.no_grad():
      torch.cuda.empty_cache()

      #test_inputs = {'input_ids': test_b_input_ids, 'token_type_ids': None, 'attention_mask': test_b_input_mask}
      test_outputs = regressor_model(test_b_input_ids, test_b_input_mask)      

      loss = loss_function(test_outputs.squeeze(), 
                             test_b_labels.squeeze())
      
      test_loss.append(loss.item())
      test_logits = test_outputs
      test_logits = test_logits.detach().cpu().numpy()
  
      test_label_ids = test_b_labels.to('cpu').numpy()
      test_predictions.append(test_logits)
      test_true_labels.append(test_label_ids)


t_all_predictions = np.concatenate(test_predictions, axis=0)
t_all_true_labels = np.concatenate(test_true_labels, axis=0)
print(t_all_predictions.shape, t_all_true_labels.shape)  #(6955, 16) (6955,) (6955,)

[3.0, 3.2, 0.2, 1.2, 3.0]
tensor([3.0000, 3.2000, 0.2000, 1.2000, 3.0000], dtype=torch.float64)
Elapsed. 0.2707679271697998
Evaling 
(20, 1) (20,)


In [ ]:
# test loss 
errs_sqd = []
for i in range(t_all_predictions.shape[0]):
  v = t_all_predictions[i][0] - t_all_true_labels[i]
  print(i,t_all_predictions[i][0], t_all_true_labels[i], v*v)
  errs_sqd.append(v*v)

print("MSE: ",sum(errs_sqd)/len(errs_sqd)  )  #MSE:  1.1414397223438868
#MSE:  1.14 for PubMedBERT Dense

0 2.9927075 3.0 5.3180687e-05
1 3.2015579 3.2 2.426825e-06
2 2.265404 0.2 4.2658935
3 2.6008215 1.2 1.9623008
4 3.6078467 3.0 0.36947766
5 2.7634246 2.0 0.5828172
6 2.6143384 1.8 0.6631471
7 2.5460465 3.4 0.7292367
8 3.5224507 3.4 0.014994147
9 1.6254601 0.2 2.0319364
10 2.248395 0.0 5.0552797
11 2.408505 2.8 0.15326832
12 2.8969805 1.6 1.6821585
13 1.6624699 0.2 2.138818
14 2.001878 1.6 0.16150592
15 3.023688 3.6 0.33213532
16 1.5471749 0.2 1.8148801
17 3.445239 3.6 0.023950918
18 2.2959163 1.8 0.24593304
19 2.7752454 2.0 0.6010055
MSE:  1.1414397223438868
Pearsons correlation: 0.812
